In [1]:
ls -al

total 16
drwxr-xr-x 1 root root 4096 Dec 18 20:29 ./
drwxr-xr-x 1 root root 4096 Dec 22 08:14 ../
drwxr-xr-x 1 root root 4096 Dec 18 20:29 .config/
drwxr-xr-x 1 root root 4096 Dec 18 20:29 sample_data/


In [0]:
!pip install konlpy
#실제로 쓰려면 느린 경우가 있어서.매캡을 깔아주려고 한다.

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!pip3 install JPype1-py3

In [0]:
# 1)매캡만 사용.
# 단어파싱이 빨라진다
# pytorch 에서 decision을 할 때, 느리다.
# 결과가 나오는 것이 마무리가 아니라. 상용화에서는 어떻게 해야하는지 알아야 한다.

In [0]:
import os
os.chdir('/tmp/')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.1.tar.gz
!tar zxfv mecab-0.996-ko-0.9.1.tar.gz
os.chdir('/tmp/mecab-0.996-ko-0.9.1')
!./configure
!make
!make check
!make install

In [0]:
!apt-get update
!apt-get install autoconf
# 업데이트를 하지 않으면 매캡을 쓸 수 없다.
# 신조어/빠른 속도/성능 /내가 할 수 있는 것이 마침표가 될 것이다.

In [0]:
#그냥 하면 root랑 자꾸 연결해야함. automake는 자동컴파일해줌.
os.chdir('/tmp')
!curl -LO http://ftpmirror.gnu.org/automake/automake-1.11.tar.gz
!tar -zxvf automake-1.11.tar.gz
os.chdir('/tmp/automake-1.11')
!./configure
!make
!make install

In [0]:
# 매캡이 깔렸다. 매캡은 일본에서 만들었기 때문에 코리아딕셔너리도 깔아야함. 그것은 아래에서
import os
os.chdir('/tmp/')


!wget -O m4-1.4.9.tar.gz http://ftp.gnu.org/gnu/m4/m4-1.4.9.tar.gz
!tar -zvxf m4-1.4.9.tar.gz
os.chdir('/tmp/m4-1.4.9')
!./configure
!make
!make install

os.chdir('/tmp')
!curl -OL http://ftpmirror.gnu.org/autoconf/autoconf-2.69.tar.gz
!tar xzf autoconf-2.69.tar.gz
os.chdir('/tmp/autoconf-2.69')
!./configure --prefix=/usr/local
!make
!make install
!export PATH=/usr/local/bin

In [0]:
import os
os.chdir('/tmp')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.0.1-20150920.tar.gz
!tar -zxvf mecab-ko-dic-2.0.1-20150920.tar.gz
os.chdir('/tmp/mecab-ko-dic-2.0.1-20150920')
!./autogen.sh
!./configure
!make
# !sh -c 'echo "dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic" > /usr/local/etc/mecabrc'
!make install

In [0]:
#매캡이 정상적으로 깔렸는지 알아보자.

os.chdir('/tmp/mecab-ko-dic-2.0.1-20150920')
!ldconfig
!ldconfig -p | grep /usr/local/lib | grep libmecab

In [0]:
# 매캡은 c나 c++베이스로 되어있다.
# 매캡과 파이썬을 연동해야한다,
# install mecab-python
import os
os.chdir('/content')

!git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git
os.chdir('/content/mecab-python-0.996')

!python3 setup.py build
!python3 setup.py install

In [0]:
cd ..

In [0]:
ls -al

In [0]:
!mkdir newsNLP

In [0]:
!mkdir newsNLP/data

In [0]:
%cd newsNLP/data

In [0]:
#데이터 다운로드
!wget http://www.kristalinfo.com/download/hkib-20000-40075.tar.gz

In [0]:
!tar xvfz hkib-20000-40075.tar.gz

In [0]:
# 리드미 확인
!cat readme.txt

In [0]:
!rm readme.txt

In [0]:
cd ..

In [0]:
!pip install torch

In [0]:
# OS 라이브러리 임포트
import os
# 정규표현식 라이브러리 임포트
import re

# scikit-learn 라이브러리 임포트
from sklearn import datasets, model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# 형태소분석기 라이브러리
from konlpy.tag import Kkma, Hannanum, Komoran, Mecab, Twitter #트위터는 이름바꼈으니 쓰지말자.

# pandas 라이브러리 임포트
import pandas as pd

# numpy 라이브러리 임포트
import numpy as np

In [0]:
dir_prefix = 'data/'
target_dir = 'HKIB-20000'
cat_dirs = ['health', 'economy', 'science', 'education', 'culture', 'society', 'industry', 'leisure', 'politics']
cat_prefixes = ['건강', '경제', '과학', '교육', '문화', '사회', '산업', '여가', '정치']

In [0]:
# 데이터 정리
files = os.listdir(dir_prefix + target_dir)

# 5분할된 텍스트 파일을 각각 처리
for file in files:
   # 데이터가 담긴 파일만 처리
   if not file.endswith('.txt'):
       continue

   # 각 텍스트 파일을 처리
   with open(dir_prefix + target_dir + '/' + file) as currfile:
       doc_cnt = 0
       docs = []
       curr_doc = None

       # 기사 단위로 분할하여 리스트를 생성
       for curr_line in currfile:
           if curr_line.startswith('@DOCUMENT'):
               if curr_doc is not None:
                   docs.append(curr_doc)
               curr_doc = curr_line
               doc_cnt = doc_cnt + 1
               continue
           curr_doc = curr_doc + curr_line

       # 각 기사를 대주제 별로 분류하여 기사별 파일로 정리
       for doc in docs:
           doc_lines = doc.split('\n')
           doc_no = doc_lines[1][9:]

           # 주제 추출
           doc_cat03 = ''
           for line in doc_lines[:10]:
               if line.startswith("#CAT'03:"):
                   doc_cat03 = line[10:]
                   break

           # 추출한 주제 별로 디렉토리 정리
           for cat_prefix in cat_prefixes:
               if doc_cat03.startswith(cat_prefix):
                   dir_index = cat_prefixes.index(cat_prefix)
                   break

           # 문서 정보를 제거하고 기사 본문만 남기기
           filtered_lines = []
           for line in doc_lines:
               if not (line.startswith('#') or line.startswith('@')):
                   filtered_lines.append(line)

           # 주제별 디렉토리에 기사를 파일로 쓰기
           filename = 'hkib-' + doc_no + '.txt'
           filepath = dir_prefix + target_dir + '/' + cat_dirs[dir_index]

           if not os.path.exists(filepath):
               os.makedirs(filepath)
           f = open(filepath + '/' + filename, 'w')
           f.write('\n'.join(filtered_lines))
           f.close()

In [0]:
# 대상이 되는 주제 폴더 선택 (교육, 건강)
dirs = ['economy', 'society'] #두가지로 주제를 분류한다. 
#dirs = ['education', 'health']

# 기사에 출현하는 단어와 레이블을 저장할 리스트를 생성
# 설명변수
x_ls = []
# 목적변수
y_ls = []

tmp1 = []
tmp2 = ''

# 형태소 분석기 객체 생성
#tokenizer_han = Hannanum()
tokenizer = Mecab()


# 각 폴더의 파일을 하나씩 읽어들이며, 전처리 후 리스트에 저장
for i, d in enumerate(dirs):
   # 파일 목록 읽어오기
   files = os.listdir(dir_prefix + target_dir + '/' + d)

   for file in files:
       # 각 파일을 읽어들이기
       f = open(dir_prefix + target_dir + '/' + d + '/' + file, 'r', encoding='utf-8')
       raw = f.read()

       # 정규표현식을 사용하여 불필요한 문자열을 제거한 다음 파일 내용을 출력
       reg_raw = re.sub(r'[-\'@#:/◆▲0-9a-zA-Z<>!-"*\(\)]', '', raw)
       reg_raw = re.sub(r'[ ]+', ' ', reg_raw)
       reg_raw = reg_raw.replace('\n', ' ')

       # 형태소분석을 거쳐 명사만을 추출한 리스트를 생성
       tokens = tokenizer.nouns(reg_raw)

       for token in tokens:
           tmp1.append(token)

       tmp2 = ' '.join(tmp1)
       x_ls.append(tmp2)
       tmp1 = []

       # 기사 주제 레이블을 리스트에 저장
       y_ls.append(i)

       # 파일 닫기
       f.close()

In [0]:
# 데이터프레임으로 변환해서 설명변수를 화면에 출력
pd.DataFrame(x_ls)

In [0]:
print(x_ls[1])

In [0]:
# 목적변수를 화면에 출력
print(y_ls)

In [0]:
# 설명변수와 목적변수를 NumPy 배열로 변환
x_array = np.array(x_ls)
y_array = np.array(y_ls)

# 단어 출현 횟수를 계수
cntvec = CountVectorizer()
x_cntvecs = cntvec.fit_transform(x_array)
x_cntarray = x_cntvecs.toarray()

# 데이터프레임으로 단어 출현 횟수 출력
pd.DataFrame(x_cntarray)

In [0]:
# 한페이지/한 장의 단어 빈도수.
# 단어의 TF-IDF 계산
tfidf_vec = TfidfVectorizer(use_idf=True)
x_tfidf_vecs = tfidf_vec.fit_transform(x_array)
x_tfidf_array = x_tfidf_vecs.toarray()

# 데이터프레임으로 변환하여 단어의 출현 횟수를 출력
pd.DataFrame(x_tfidf_array)

In [0]:
# 데이터를 훈련 데이터와 테스트 데이터로 분할
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(x_tfidf_array, y_array, test_size=0.2)

# 데이터 건수 확인
print(len(train_X))
print(len(test_X))

In [0]:
# 전처리완료

In [0]:
# PyTorch 라이브러리 임포트
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [0]:
# 훈련 데이터 텐서 생성
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

# 테스트 데이터 텐서 생성
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

# 텐서로 변환한 데이터 건수 확인
print(train_X.shape)
print(train_Y.shape)

In [0]:
print(train_X.shape[1]) #37568개.

In [0]:
# 파이토치에서는 텐서 데이터 셋을 쓴다. 그것을 위해서 작업을 해야함.
# 설명변수와 목적변수의 텐서를 합친다
train = TensorDataset(train_X, train_Y)

# 첫번째 텐서 내용 확인
print(train[0])

# 미니배치 분할하기
train_loader = DataLoader(train, batch_size=100, shuffle=True)

In [0]:
# 이제 신경망을 구성해야함. 뉴럴네트워크
# 신경망 구성
class Net(nn.Module):
   def __init__(self):
       super(Net, self).__init__()
       self.fc1 = nn.Linear(train_X.shape[1], 256) # 37568개 input
       self.fc2 = nn.Linear(256, 256) #256씩 커넥티드
       self.fc3 = nn.Linear(256, 256)
       self.fc4 = nn.Linear(256, 128)
       self.fc5 = nn.Linear(128, 128)
       self.fc6 = nn.Linear(128, len(dirs)) #카테고리분류. len(dirs)위에서 확인할 것. 

   def forward(self, x):
       x = F.relu(self.fc1(x))
       x = F.relu(self.fc2(x))
       x = F.relu(self.fc3(x))
       x = F.relu(self.fc4(x))
       x = F.relu(self.fc5(x))
       x = self.fc5(x)
       return F.log_softmax(x)

# 인스턴스 생성
model = Net()

In [0]:
#training

# 오차함수
criterion = nn.CrossEntropyLoss()

# 최적화 기법 선택
optimizer = optim.Adam(model.parameters(), lr=0.005)

# 학습
for epoch in range(20):
   total_loss = 0

   # 분할한 데이터(미니배치)를 차례로 꺼내옴
   for train_x, train_y in train_loader:
       # 계산 그래프 구성
       train_x, train_y = Variable(train_x), Variable(train_y)
       # 경사 초기화
       optimizer.zero_grad()
       # 순전파 계산
       output = model(train_x)
       # 오차 계산
       loss = criterion(output, train_y)
       # 역전파 계산
       loss.backward()
       # 가중치 업데이트
       optimizer.step()
       # 오차 누적 계산

       total_loss += loss.item()

   # 100 에포크마다 누적오차를 출력
   if (epoch+1) % 1 == 0:
       print(epoch+1, total_loss)

In [0]:
# 오래걸린다.
# 시간을 단축시키는 방법을 찾아내어야 한다.
# work가 끝났다는 것은 그런 시간까지도 단축해야한다는 것을 뜻한다.

# 계산 그래프 구성
test_x, test_y = Variable(test_X), Variable(test_Y)

# 출력이 0 혹은 1이 되도록
result = torch.max(model(test_x).data, 1)[1]

# 모형의 정확도 계산
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

# 정확도 출력
accuracy

In [0]:
# 97% 